# Damage Size Regression on Synthetic Data - Baseline
In this notebook, we conduct experiments to be able to compare to a baseline. Just to make sure, we repeat the experiment several times to make sure that the previously established cross-validation setup gives reliable results also with this approach.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/berni-lehner/structural_health_monitoring/blob/main/notebooks/synthetic_regression_AA_BASE.ipynb)

In [1]:
%load_ext rich
%load_ext autoreload
%autoreload 2
%matplotlib inline

from rich import print

random_state = 42

## Prerequisites

In [2]:
import sys
import os
from pathlib import Path
import pandas as pd

# initialize environment, paths, etc...
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # uncomment in case you want to start from scratch in Colab
    ! rm -rf /content/structural_health_monitoring  
    ! git clone https://github.com/berni-lehner/structural_health_monitoring.git
    % pip install -r /content/structural_health_monitoring/requirements.txt

    # this is Colab specific, and only a manual "Run all" will cause the
    # installation to be finished
    if pd.__version__ != "1.4.2":
        print("Stopping RUNTIME! Please run again to finish installation.")
        os.kill(os.getpid(), 9)

    DATA_PATH = Path(r"/content/structural_health_monitoring/data")
    sys.path.insert(0, "/content/structural_health_monitoring/src")
else:
    DATA_PATH = Path(r"../data/")
    sys.path.insert(0, "../src")


sys.path.insert(0, str(DATA_PATH))
SYN_DATA_PATH = Path(DATA_PATH, "synthetic")
REAL_DATA_PATH = Path(DATA_PATH, "real_world")
RESULT_PATH = Path(DATA_PATH, "results")
if not os.path.isdir(RESULT_PATH):
    os.makedirs(RESULT_PATH)

try:
    from zippeekiyay import namelist
except ImportError or ModuleNotFoundError:
    print("installing zippee-ki-yay...")
    % pip install git+https://github.com/berni-lehner/zippee-ki-yay.git

    from zippeekiyay import namelist

CALIBRATION_FILE = Path(REAL_DATA_PATH, "calibration.pkl")

In [3]:
# Intel(R) Extension for Scikit-learn for speed-up
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import time
import glob
import sklearn
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

# ...and download raw data if not already downloaded
from data_utils import init_data

init_data(syn_data_path=SYN_DATA_PATH)

from plot_utils import (
    init_plotting,
    plot_metrics,
    plot_cv_indices,
    plot_embedding_targets,
    init_plotting,
    plot_classwise_dist,
    plot_classwise_kde,
)

# configure fonts, etc...
init_plotting()

from scoring_utils import get_synth_regression_scoring, SHM_Scoring
from cv_utils import (
    AnomalyShuffleSplit,
    RepeatedAnomalyShuffleSplit,
    CreateAnomalyData,
    dump_cv,
)
from bench_utils import classification_benchmark, repeat_experiment, extract_metrics
from data_utils import load_syn_reg_data, FEATURE_LIST
from shm_experiments import conduct_aa_reg_experiment

time passed: 6.75 s
downloading synthetic data successful: True


In [5]:
result_file = "synthetic_regression_AA_hopfield.pkl"
result_file = Path(RESULT_PATH, result_file)

## Data Loading

In [6]:
X, y = load_syn_reg_data(
    data_path=SYN_DATA_PATH, target_col=["y_radius"], min_radius=1.9, cache=True
)

In [7]:
X.shape

(28302, 31)

In [8]:
cnt = Counter(y)
cnt


Counter({
    0.0: 990,
    2.2: 979,
    2.5: 979,
    2.8: 978,
    3.1: 986,
    3.4: 972,
    3.8: 973,
    4.1: 976,
    4.4: 979,
    4.7: 976,
    5.0: 977,
    6.7: 981,
    8.3: 972,
    10.0: 973,
    12.0: 969,
    14.0: 978,
    16.0: 980,
    18.0: 976,
    20.0: 974,
    22.0: 975,
    24.0: 980,
    26.0: 973,
    28.0: 983,
    30.0: 977,
    32.0: 967,
    34.0: 971,
    36.0: 976,
    38.0: 964,
    40.0: 968
})

## A-A Experiments

In [9]:
from imblearn.under_sampling import ClusterCentroids
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder

samples_per_class = 40
classes, counts = np.unique(y, return_counts=True)
n_classes = len(classes)
strgy = dict(zip(range(n_classes), [samples_per_class] * n_classes))

sampler = ClusterCentroids(
    estimator=KMeans(n_init="auto"),
    sampling_strategy=strgy,
    random_state=random_state,
)

enc = OrdinalEncoder(dtype=np.int32)

y_cls = enc.fit_transform(y.reshape(-1, 1))
X_sampled, y_recls = sampler.fit_resample(X, y_cls)
y_sampled = enc.inverse_transform(y_recls.reshape(-1, 1))  # .squeeze()


X_sampled.shape, y_sampled.shape

((1160, 31), (1160, 1))

In [12]:
import torch
from imblearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
from hflayers import Hopfield, HopfieldLayer

In [30]:
beta = 1.0
hopfield = Hopfield(
    scaling=beta,
    # do not project layer input
    state_pattern_as_static=True,
    stored_pattern_as_static=True,
    pattern_projection_as_static=True,
    # do not pre-process layer input
    normalize_stored_pattern=False,
    normalize_stored_pattern_affine=False,
    normalize_state_pattern=False,
    normalize_state_pattern_affine=False,
    normalize_pattern_projection=False,
    normalize_pattern_projection_affine=False,
    # do not post-process layer output
    disable_out_projection=True,
)

estimator = NeuralNetRegressor(hopfield)

pipeline = Pipeline([
    ("to_tensor"),    
    ("estimator", estimator)
])

k = torch.from_numpy(X[np.newaxis])
q = torch.from_numpy(X_sampled[np.newaxis])
v = torch.from_numpy(y_sampled[np.newaxis]).float()

print(q.shape, k.shape, v.shape)
hopfield((q, k, v))

torch.Size([1, 1160, 31])
torch.Size([1, 28302, 31])
torch.Size([1, 1160, 1])


tensor([[[34.0021],
         [34.0019],
         [34.0016],
         ...,
         [40.0000],
         [40.0000],
         [40.0000]]])

In [14]:
scoring = get_synth_regression_scoring()

results = conduct_aa_reg_experiment(
    X=X,
    y=y,
    estimator=pipeline,
    name="BASE",
    n_repeats=5,
    n_splits=32,
    test_size=0.1,
    scoring=scoring,
    random_state=random_state,
)


All the 32 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32 fits failed with the following error:
Traceback (most recent call last):
  File "/home/huberc/sw/mambaforge/envs/quick/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/huberc/sw/mambaforge/envs/quick/lib/python3.9/site-packages/imblearn/pipeline.py", line 297, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "/home/huberc/sw/mambaforge/envs/quick/lib/python3.9/site-packages/skorch/regressor.py", line 82, in fit
    return super(NeuralNetRegressor, self).fit(X, y, **fit_params)
  File "/home/huberc/sw/mambaforge/envs/quick/lib/python3.9/site-packages/skorch/net.py", line 1319, in fit
   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 scoring = get_synth_regression_scoring()                                                    │
│    2                                                                                             │
│ ❱  3 results = conduct_aa_reg_experiment(                                                        │
│    4 │   X=X,                                                                                    │
│    5 │   y=y,                                                                                    │
│    6 │   estimator=pipeline,                                                                     │
│                                                                                                  │
│ /home/huberc/repos/quick/structural_health_monitoring/notebooks/../src/shm_experiments.py:117 in │
│ conduct_aa_reg_experiment                                                                        │
│                                                                                                  │
│   114 │   │   for i in range(n_repeats)                                                          │
│   115 │   ]                                                                                      │
│   116 │                                                                                          │
│ ❱ 117 │   results = regression_benchmark(                                                        │
│   118 │   │   X=X, y=y, models=estimators, cv=cv, scoring=scoring, random_state=random_state     │
│   119 │   )                                                                                      │
│   120                                                                                            │
│                                                                                                  │
│ /home/huberc/repos/quick/structural_health_monitoring/notebooks/../src/bench_utils.py:94 in      │
│ regression_benchmark                                                                             │
│                                                                                                  │
│    91 │   │   │   "neg_mean_squared_error",                                                      │
│    92 │   │   ]                                                                                  │
│    93 │                                                                                          │
│ ❱  94 │   results = benchmark(                                                                   │
│    95 │   │   X=X,                                                                               │
│    96 │   │   y=y,                                                                               │
│    97 │   │   models=models,                                                                     │
│                                                                                                  │
│ /home/huberc/repos/quick/structural_health_monitoring/notebooks/../src/bench_utils.py:165 in     │
│ benchmark                                                                                        │
│                                                                                                  │
│   162 │   │   │   except ValueError as exc:                                                      │
│   163 │   │   │   │   print(exc)                                                                 │
│   164 │   │   │                                                                                  │
│ ❱ 165 │   │   │   tmp = pd.DataFrame(cv_results)                                                 │
│   166 │   │   │   tmp["model"] = name                      

## Store results for later

In [15]:
results.to_pickle(result_file)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 results.to_pickle(result_file)                                                               │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'results' is not defined

## Plot some results for overview

In [16]:
fig = plot_metrics(
    extract_metrics(
        results, ["test_mse_0.0_40.0", "test_mse_2.8_40.0", "test_mse_5.0_40.0"]
    )
)
plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 fig = plot_metrics(                                                                          │
│   2 │   extract_metrics(                                                                         │
│ ❱ 3 │   │   results, ["test_mse_0.0_40.0", "test_mse_2.8_40.0", "test_mse_5.0_40.0"]             │
│   4 │   )                                                                                        │
│   5 )                                                                                            │
│   6 plt.show()                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'results' is not defined

In [17]:
shm_scoring = SHM_Scoring()

fig = plot_metrics(extract_metrics(results, shm_scoring.SYNTH_MSE_RESULTS))
plt.gca().get_legend().remove()

plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 shm_scoring = SHM_Scoring()                                                                  │
│   2                                                                                              │
│ ❱ 3 fig = plot_metrics(extract_metrics(results, shm_scoring.SYNTH_MSE_RESULTS))                  │
│   4 plt.gca().get_legend().remove()                                                              │
│   5                                                                                              │
│   6 plt.show()                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'results' is not defined